***
# PVA Function
### _Calculates Present Value Amount_
> _support function used by the SecValue Function_<br>

In [1]:
def Fx_PVA(apr,numOfPeriods,payment, frequency):
    apr =round(apr,4)
    payment = round(payment,2)
    rate = apr / (frequency * 100)
    
    if rate == 0:
        return numOfPeriods * payment
    else:
        return payment * ((1 - 1/((1+rate)**numOfPeriods))/rate)

***
***
# SecValue Function
### _Calculates the Securitization value of a Lease_
Fx_SecValueMENM(discountRate, remainingTerm , paymentDue, payAheadAmtorDelinqAmt,residual,delinquencyType = 'First',frequency=12):
    <br>

The parameters are:
- discountRate (Discount Rate) = S_InvestorRate
- remainingTerm (Remaining Term) = S_CurrRemNbrPmts 
- paymentDue (Rental Payment Due) =  O_BaseRentPmt
- payAheadAmtorDelinqAmt (PayAhead or Delinquent Amount) =  O_BaseRentRecBal
- residual (Residual Value) = S_InvResidual
- delinquencyType = 'First'
- frequency = 12


<div class="alert alert-block alert-success">
<b> The function calculates four intermediate discounted components: </b>
<br>
<b>1. pvaPart (Present Value of future rent payments)</b> = Present Value of future scheduled payment factoring any payahead amount.<br>
<b>2. delinqPart (Present Value of delinquent payments) </b> = Present Value of the delinquent payment.<br>
<i>Note:At TFS the delinquent amount will be discounted by 1 period as it is assumed that it will be collected in the next period. Other issues may assume the delinquent amounts will be collected at the end of the term.</i><br>
<b>3. partialPmtPart (Present Value of partial payahead rent payments) </b>= Present Value of a partial payahead. <br>
<i>For example if the customer's payment is 500 and RemTerm is 6, but we have payahead of 1150, that means there are 2 full prepayments and one partial prepayment of 150. The 150 will be discounted with in the 3rd period i.e the next period following the 2 full prepayments.</i><br>
<b>4. residualPmtPart (Present Value of the residual value) </b><br>
   
<b>The SecValue is sum of: pvaPart + delinqPart + partialPmtPart + residualPmtPart </b> 
 
 <div>

In [2]:
def Fx_SecValueMENM(discountRate, remainingTerm , paymentDue, payAheadAmtorDelinqAmt,residual,delinquencyType = 'First',frequency=12):
    
    discountRate = round(discountRate,4)

######################### If the Remaining Term is 0 or less then Calculate SecValue ########################   
    if remainingTerm < 1:
        print((residual - payAheadAmtorDelinqAmt)/(1.0 + discountRate / (frequency * 100)))

### Calculate the Delinquency/Payahead period. Used in the delinqPart.   
    if delinquencyType == 'Last':
        if remainingTerm - 1 > 0:
            delinqPeriod = remainingTerm - 1
        else:
            delinqPeriod = 0
    else:
        delinqPeriod = 1

### Convert the annual discount rate to effective monthly rate        
    rate = discountRate / (frequency * 100)

### Calculate the number of full prepayments made and partial payahead amount if any.
#*Note: The number full prepayments would result in no payment due for that many periods.The partial payahead amount will be also discounted accordingly and factored in the final SecValue result
    
    if paymentDue == 0: #Note: This seem to NOT be possible. And has never happened, so the logic was dormant.
        nbrFullPrePmts = 0
        partialPayment = -payAheadAmtorDelinqAmt
    else:
        nbrFullPrePmts = payAheadAmtorDelinqAmt/ paymentDue if payAheadAmtorDelinqAmt / paymentDue > 0 else 0
        nbrFullPrePmts = int(nbrFullPrePmts)
        partialPayment = paymentDue - (payAheadAmtorDelinqAmt - (nbrFullPrePmts*paymentDue)) if payAheadAmtorDelinqAmt - (nbrFullPrePmts*paymentDue) > 0.005 else 0

    nbrFullPrePmts = int(nbrFullPrePmts)
    print ('Number of Full Prepayments',nbrFullPrePmts)
    print ('Partial PayAhead Amount',round(partialPayment,2))

    if partialPayment > 0:
        partialPmtFlag = 1
    else:
        partialPmtFlag = 0

######################### 1. pvaPart
    if (nbrFullPrePmts + partialPmtFlag) != 0:
        pvaPart = Fx_PVA(discountRate,(remainingTerm - (nbrFullPrePmts + partialPmtFlag) - 1 if (remainingTerm - (nbrFullPrePmts + partialPmtFlag) - 1) > 0 else 0), paymentDue, frequency) / ((1 + rate)**(nbrFullPrePmts + partialPmtFlag))
        #print('f+p>0',pvaPart)
    else:
        pvaPart = Fx_PVA(discountRate,(remainingTerm - (nbrFullPrePmts + partialPmtFlag) - 1 if (remainingTerm - (nbrFullPrePmts + partialPmtFlag) - 1) > 0 else 0), paymentDue, frequency)
        #print('f+p>0',pvaPart)
        
    pvaPart = round(pvaPart,10)

####################### 2. delinqPart       
    delinqPart = (-payAheadAmtorDelinqAmt if -payAheadAmtorDelinqAmt > 0 else 0) / ((1 + rate)**delinqPeriod)

####################### 3. partialPmtPart  
    #IF the customer has pre-paid more than the remaining number of payments, then there is no future payment
    if ((((remainingTerm -1)* paymentDue) - payAheadAmtorDelinqAmt) <= 0):
        partialPmtPart = 0
    #Else discount the partial Payment Amount
    else:
        partialPmtPart = partialPayment /((1 + rate)**(nbrFullPrePmts + 1))

####################### 4. residualPmtPart
    #Recalculate the residual if the customer has paid more than the reminaing number of monthly payments
    TempCalc = - payAheadAmtorDelinqAmt if (remainingTerm - 1)* paymentDue < 0 else (remainingTerm - 1)* paymentDue - payAheadAmtorDelinqAmt 
    #calculate the effective residual value to be discounted
    effectiveResidual = max(0,residual + (TempCalc if TempCalc < 0 else 0))
    #calculate the residualPmtPart
    residualPmtPart = effectiveResidual / ((1 + rate)**remainingTerm)
    

    print ('\n','pvaPart:',round(pvaPart,2),'\n',
           'delinqPart:', round(delinqPart,2),'\n',
           'partialPmtPart:',round(partialPmtPart,2),'\n',
           'residualPmtPart:',round(residualPmtPart,2),'\n',
           'SecValue:',round(pvaPart + delinqPart + partialPmtPart + residualPmtPart,2))


***
***
***
# SecValue Examples

***
***
**PullAhead Scenario with (incorrect) Delinquent Amount**
<br> 1. The remainingTerm (S_CurrRemNbrPmts) in the SecValue function for PullAhead is hardcoded to 1 and paymentDue (O_BaseRentPmt) is hardcoded to 0, since for a PullAhead it is assumed there are no more lease payments expected/due from the customer and the residual would be discounted by 1 period.
<br> 2. O_BaseRentRecBal as usual represents either a payAhead amount or Delinquent amount. It is a:
> - Payahead (if the value is negative) 
> - Delinquent Payment (if the value positive)       

<div class="alert alert-block alert-info">

The problem with having a Delinquent Amount for a Pullahead is that the SecValue functions has logic:
> - If the rent Payment Due (hardcoded to 0 for PullAheads) = 0 then the full delinquent payment (O_BaseRentRecBal) is incorrectly considered as partial payment. And it doesn't seem that we have ever had a scenario where an account has had O_BaseRentPmt = 0 and O_BaseRentRecBal <> 0, so that part of the logic was always dormant.
> - The delinquent payment (O_BaseRentRecBal) is also considered as delinquent and thus doublecounted
</div>

**Inputs to SecValue**
<br>
remainingTerm is ignored and hardcoded to 1 in the SecValue function #S_CurrRemNbrPmts<br>
discountRate = 6.05000  #S_InvestorRate<br>
paymentDue is ignored and hardcoded to 0 in the SecValue function0 #O_BaseRentPmt<br>
payAheadAmtorDelinqAmt =  1364.28 #O_BaseRentRecBal<br>
residual =  22657.00 #S_InvResidual<br>

In [3]:
###Inputs to SecValue####

#remainingTerm is ignored and hardcoded to 1 in the SecValue function #S_CurrRemNbrPmts
discountRate = 6.05000  #S_InvestorRate
#paymentDue is ignored and hardcoded to 0 in the SecValue function0 #O_BaseRentPmt
payAheadAmtorDelinqAmt =  1364.28 #O_BaseRentRecBal
residual =  22657.00 #S_InvResidual
print(Fx_SecValueMENM(discountRate, 1, 0,-payAheadAmtorDelinqAmt,residual, 'First',12))

Number of Full Prepayments 0
Partial PayAhead Amount 1364.28

 pvaPart: 0.0 
 delinqPart: 1357.44 
 partialPmtPart: 1357.44 
 residualPmtPart: 22543.34 
 SecValue: 25258.22
None


***
***
**PullAhead Scenario with (incorrect) Delinquent Amount**
<br> 1.The remainingTerm (S_CurrRemNbrPmts) in the SecValue function for PullAhead is hardcoded to 1 and paymentDue (O_BaseRentPmt) is hardcoded to 0, since for a PullAhead it is assumed there are no more lease payments expected/due from the customer and the residual would be discounted by 1 period.

<br> 2. O_BaseRentRecBal as usual represents either a PayAhead amount or Delinquent amount. It is a:
> - Payahead (if the value is negative) 
> - Delinquent Payment (if the value positive)

**Inputs to SecValue**
<br>
remainingTerm is ignored and hardcoded to 1 in the SecValue function #S_CurrRemNbrPmts<br>
discountRate = 6.05000  #S_InvestorRate<br>
paymentDue is ignored and hardcoded to 0 in the SecValue function0 #O_Base<br>
payAheadAmt =  0 #O_BaseRentRecBal<br>
residual =  22657.00 #S_InvResidual<br>

In [4]:
###Inputs to SecValue####

#remainingTerm is ignored and hardcoded to 1 in the SecValue function #S_CurrRemNbrPmts
discountRate = 6.05000  #S_InvestorRate
#paymentDue is ignored and hardcoded to 0 in the SecValue function0 #O_Base
payAheadAmt =  0 #O_BaseRentRecBal
residual =  22657.00 #S_InvResidual
print(Fx_SecValueMENM(discountRate, 1, 0,-payAheadAmt,residual, 'First',12))

Number of Full Prepayments 0
Partial PayAhead Amount 0

 pvaPart: 0.0 
 delinqPart: 0.0 
 partialPmtPart: 0 
 residualPmtPart: 22543.34 
 SecValue: 22543.34
None


***
***
**Active and Not Delinquent and Not PayAhead**

**Inputs to SecValue**
<br>
remainingTerm = 6 # S_CurrRemNbrPmts<br>
discountRate = 6.05000  #S_InvestorRate<br>
paymentDue =   454.76 #O_BaseRentPmt<br>
payAheadAmtorDelinqAmt =  0 #O_BaseRentRecBal<br>
residual =  22657.00 #S_InvResidual<br>

In [5]:
###Inputs to SecValue####

remainingTerm = 6 # S_CurrRemNbrPmts
discountRate = 6.05000  #S_InvestorRate
paymentDue =   454.76 #O_BaseRentPmt
payAheadAmtorDelinqAmt =  0 #O_BaseRentRecBal
residual =  22657.00 #S_InvResidual
print(Fx_SecValueMENM(discountRate, remainingTerm +1, paymentDue,-payAheadAmtorDelinqAmt,residual, 'First',12))
   

Number of Full Prepayments 0
Partial PayAhead Amount 0

 pvaPart: 2681.05 
 delinqPart: 0.0 
 partialPmtPart: 0.0 
 residualPmtPart: 21873.28 
 SecValue: 24554.33
None


***
***
**Active and PayAhead**
<br> _the account has a full payment ahead and partial payahead amount_

**Inputs to SecValue**
<br>
remainingTerm = 6 # S_CurrRemNbrPmts<br>
discountRate = 5.95  #S_InvestorRate<br>
paymentDue =   306.54 #O_BaseRentPmt<br>
payAheadAmtorDelinqAmt =  -577.57 #O_BaseRentRecBal<br>
residual =  18737.00 #S_InvResidual<br>

In [6]:
###Inputs to SecValue####

remainingTerm = 6 # S_CurrRemNbrPmts
discountRate = 5.95  #S_InvestorRate
paymentDue =   306.54 #O_BaseRentPmt
payAheadAmtorDelinqAmt =  -577.57 #O_BaseRentRecBal
residual =  18737.00 #S_InvResidual
print(Fx_SecValueMENM(discountRate, remainingTerm +1, paymentDue,-payAheadAmtorDelinqAmt,residual, 'First',12))


Number of Full Prepayments 1
Partial PayAhead Amount 35.51

 pvaPart: 1199.19 
 delinqPart: 0.0 
 partialPmtPart: 35.16 
 residualPmtPart: 18099.38 
 SecValue: 19333.73
None


***
***
**Active and Delinquent**
<br> _the account has more than one delinquent payments_

**Inputs to SecValue**
<br>
remainingTerm = 6 # S_CurrRemNbrPmts<br>
discountRate = 5.5  #S_InvestorRate<br>
paymentDue =   397.2 #O_BaseRentPmt<br>
payAheadAmtorDelinqAmt =  758.25 #O_BaseRentRecBal<br>
residual =  22545.00 #S_InvResidual<br>

In [7]:
remainingTerm = 6 # S_CurrRemNbrPmts
discountRate = 5.5  #S_InvestorRate
paymentDue =   397.2 #O_BaseRentPmt
payAheadAmtorDelinqAmt =  758.25 #O_BaseRentRecBal
residual =  22545.00 #S_InvResidual
print(Fx_SecValueMENM(discountRate, remainingTerm +1, paymentDue,-payAheadAmtorDelinqAmt,residual, 'First',12))

Number of Full Prepayments 0
Partial PayAhead Amount 0

 pvaPart: 2345.43 
 delinqPart: 754.79 
 partialPmtPart: 0.0 
 residualPmtPart: 21834.76 
 SecValue: 24934.98
None


In [9]:
print(Fx_SecValueMENM(0.0525000000,18, 592.105,0,0, 'First',12))

Number of Full Prepayments 0
Partial PayAhead Amount 0

 pvaPart: 10061.91 
 delinqPart: 0.0 
 partialPmtPart: 0.0 
 residualPmtPart: 0.0 
 SecValue: 10061.91
None


In [15]:
Fx_PVA(5.25000000,18,592.105, 12)

10227.636592836896